# Data cleaning

*Based on DA and CCS1 materials*

In this notebook:

* Inspection of dataframes
* Identification and handling of missing values
* Usage of functions to clean columns
* Merging dataframes


* .describe(), .dtypes
* .isna().sum(), .fillna(), . dropna()
* .apply() and selection of functions based on existing list
* .merge()

In [ ]:
import pandas as pd
import numpy as np

## Loading data

Two datasets:
* YouTube videos on climate change
* Sentiment analysis on titles of these videos (three scores, on Thursday you will learn what is wrong with that ;))

### What do we want to know?

* To what extent does the sentiment expressed in the title of the video influence user engagement (views, likes and dislikes)?
* To what extent does the sentiment expressed in the title of the video vary depending on the category in which the video is published?

**Important:** We only want to do this for videos published in 2018 till 2020. 

What do we need to do
1. **Inspect** datasets
2. Make sure we have the user engagement variables (**likes**, **dislikes**, and **views**)
3. Make sure we have a variable for the **category** of the video
4. Know **when the video was published**, and remove old videos
5. **Merge** the two datasets

In [ ]:
videos = pd.read_csv('YouTube_climatechange.tab')

In [ ]:
videos = pd.read_csv('YouTube_climatechange.tab', sep='\t')
videos

## Data exploration

In [ ]:
len(videos)

In [ ]:
videos.head()

In [ ]:
videos.columns

In [ ]:
videos.describe()

In [ ]:
videos.isna().sum()

# Data cleaning

### Missing values

In [ ]:
videos.isna().sum()

In [ ]:
videos[['likeCount', 'dislikeCount']].describe()

The minimum value is 0, so probably something else is going on. Let's see if this is related to channels (e.g., some channels not allowing users to like videos, perhaps?).

In [ ]:
videos[videos['likeCount'].isna()]['channelTitle'].value_counts()

In [ ]:
videos['channelTitle'].value_counts()

In [ ]:
channels_with_likes = videos[videos['likeCount'].isna()==False]['channelTitle'].unique().tolist()

In [ ]:
channels_with_likes

Using "in" operator we can check if an element is present in a list.

In [ ]:
'Intergovernmental Panel on Climate Change (IPCC)' in channels_with_likes

In [ ]:
'The Lancet' in channels_with_likes

If I wanted to substitute them by another value, I would use the following command:

In [ ]:
#Fill NA's with 0
videos['dislikeCount_no_na'] = videos['dislikeCount'].fillna(0)

In [ ]:
videos.isna().sum()

In [ ]:
#Drop rows with NA
videos = videos.dropna(subset=['likeCount', 'dislikeCount'])

In [ ]:
videos.isna().sum()

In [ ]:
len(videos)

## Filter on date - "recoding" a column

In [ ]:
videos['publishedAt']

From my key variables so far (likeCount, dislikeCount, videoTitle, videoCategoryLabel), all looks OK. The numeric variables are in numeric form (int or float), and the text variables are in object form.

But the date variable (publishedAt) is stored as an object... and it should be a date.

In [ ]:
videos['publishedAt'].head()

In [ ]:
#Apply a funcation to a column
videos['publishedAt'] = videos['publishedAt'].apply(pd.to_datetime)

In [ ]:
videos['publishedAt'].head()

In [ ]:
videos.dtypes

Now we can filter based on publication date

In [ ]:
videos[videos['publishedAt'] >'2018-12-31']

In [ ]:
videos['publishedAt'].min()

In [ ]:
videos['publishedAt'].max()

In [ ]:
len(videos)

In [ ]:
len(videos[videos['publishedAt'] >'2017-12-31'])

In [ ]:
videos = videos[videos['publishedAt'] >'2017-12-31']

## Operations on columns

### Changing data type


In [ ]:
sentiment = pd.read_csv('Sentiment_YouTubeClimateChange.csv')

In [ ]:
#Disclaimer - very crude measure, more on it on Thursday!
sentiment.head()

In [ ]:
sentiment.dtypes

In [ ]:
sentiment.describe()

All columns are objects. Do we need to do anything with that?

In [ ]:
sentiment.isna().sum()

In [ ]:
sentiment["negative"] = pd.to_numeric(sentiment["negative"])
sentiment["positive"] = pd.to_numeric(sentiment["positive"])

In [ ]:
sentiment['overall'] = sentiment['negative'] + sentiment['positive']

In [ ]:
sentiment['overall'].describe()

### Applying functions and recoding categorical variables

In [ ]:
videos['videoCategoryLabel'].value_counts()

In [ ]:
def recategorize(category):
    if category == 'News & Politics':
        return category
    if category == 'Education':
        return 'Education, Science and Technology'
    if category == 'Science & Technology':
        return 'Education, Science and Technology'
    if category == 'Nonprofits & Activism':
        return category
    else:
        return 'Other'
    

In [ ]:
videos['category'] = videos['videoCategoryLabel'].apply(recategorize)

In [ ]:
videos['category'].value_counts()

In [ ]:
videos[['category', 'favoriteCount', 'commentCount', 'dislikeCount_no_na']].groupby('category').agg(np.mean)

In [ ]:
videos[['commentCount','category']].groupby('category').agg(np.mean).plot(kind='bar')

In [ ]:
videos.columns

# Afternoon

## Merging the dataframes

A few important tips:
* Make sure that you have one unique identifier (column) that is available in both dataframes
* Make sure that the unique identifier column has the same name in both dataframes
* Make sure that the unique identifier column is of the same data type in both dataframes

In [ ]:
videos.columns

In [ ]:
sentiment['videoId'].value_counts()

In [ ]:
sentiment.columns

In [ ]:
videos.dtypes

In [ ]:
sentiment.dtypes

OK, the column is available in both, and seems to be of the same data type. So I can merge.

In [ ]:
len(videos)

In [ ]:
len(sentiment)

In [ ]:
merge1 = videos.merge(sentiment, on='videoId', how='right')

In [ ]:
merge1.isna().sum()

In [ ]:
merge2 = videos.merge(sentiment, on='videoId', how='left')
merge2.isna().sum()

In [ ]:
len(videos.merge(sentiment, on='videoId', how='right'))

In [ ]:
merge3 = videos.merge(sentiment, on='videoId', how='inner')
merge3.isna().sum()

In [ ]:
len(videos.merge(sentiment, on='videoId', how='outer'))

In [ ]:
videos_sent = videos.merge(sentiment, on='videoId')

In [ ]:
len(videos_sent)

## Writing the merged file

In [ ]:
videos_sent.to_pickle('videos_sent.pkl')

## Aggregation

In [ ]:
videos[['videoCategoryLabel', 'commentCount']].groupby('videoCategoryLabel').agg([np.mean, sum])

In [ ]:
videos[['commentCount','videoCategoryLabel']].groupby('videoCategoryLabel').agg(np.mean).plot(kind='bar')

In [ ]:
videos.columns

In [ ]:
videos['publishedAt']

In [ ]:
#Create a new column for month
videos['month'] = videos['publishedAt'].dt.strftime('%Y-%m-%d')


In [ ]:
videos[['commentCount','month']].groupby('month').agg(sum).plot()

In [ ]:
videos[['commentCount','month']].groupby('month').agg(np.mean)